In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
from config import *

In [ ]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, shape=[batch_size, N])
y = tf.placeholder(tf.float32, shape=[batch_size, K])

net_dict = {}
bp_iter_num = 5
RNN = 1

inf_num = 1000
    
# initial
for i in range(n+1):
    for j in range(N):
        net_dict["L_{0}{1}{2}".format(i,j,0)] = tf.zeros((batch_size))
        net_dict["R_{0}{1}{2}".format(i,j,0)] = tf.zeros((batch_size))

if(RNN):
    LV = tf.Variable(np.float32(np.ones((n,N,1))))
    RV = tf.Variable(np.float32(np.ones((n,N,1))))
else:
    LV = tf.Variable(np.float32(np.ones((n,N,bp_iter_num))))
    RV = tf.Variable(np.float32(np.ones((n,N,bp_iter_num))))

for j in range(N):
    net_dict["L_{0}{1}{2}".format(n,j,0)] = tf.ones((1))*x[:,j]    
    if(FZlookup[j] == 0):
        net_dict["R_{0}{1}{2}".format(0,j,0)] = tf.ones((batch_size))*inf_num

loss = 0

# bp algorithm
for k in range(bp_iter_num):
    if(RNN):
        itr = 0
    else:
        itr = k
    for i in range(n,0,-1):
        for phi in range(2**i):
            psi = int(np.floor(phi/2))
            if(np.mod(phi,2)!=0):
                for omega in range(2**(n-i)):
                    net_dict["R_{0}{1}{2}".format(n+1-i,psi+2*omega*2**(i-1),0)] = RV[n-i,psi+2*omega*2**(i-1),itr]*fFunction(net_dict["L_{0}{1}{2}".format(n+1-i,psi+(2*omega+1)*2**(i-1),0)]+net_dict["R_{0}{1}{2}".format(n-i,phi+omega*2**i,0)], net_dict["R_{0}{1}{2}".format(n-i,phi-1+omega*2**i,0)])
                    net_dict["R_{0}{1}{2}".format(n+1-i,psi+(2*omega+1)*2**(i-1),0)] = net_dict["R_{0}{1}{2}".format(n-i,phi+omega*2**i,0)]+RV[n-i,psi+(2*omega+1)*2**(i-1),itr]*fFunction(net_dict["L_{0}{1}{2}".format(n+1-i,psi+(2*omega)*2**(i-1),0)],net_dict["R_{0}{1}{2}".format(n-i,phi-1+omega*2**i,0)])
    for i in range(1,n+1):
        for phi in range(2**i):
            psi = int(np.floor(phi/2))
            for omega in range(2**(n-i)):
                if(np.mod(phi,2)==0):
                    net_dict["L_{0}{1}{2}".format(n-i,phi+omega*2**i,0)] = LV[n-i,phi+omega*2**i,itr]*fFunction(net_dict["L_{0}{1}{2}".format(n+1-i,psi+2*omega*2**(i-1),0)],net_dict["L_{0}{1}{2}".format(n+1-i,psi+(2*omega+1)*2**(i-1),0)]+net_dict["R_{0}{1}{2}".format(n-i,phi+1+omega*2**i,0)])             
                else:
                    net_dict["L_{0}{1}{2}".format(n-i,phi+omega*2**i,0)] = net_dict["L_{0}{1}{2}".format(n+1-i,psi+(2*omega+1)*2**(i-1),0)]+LV[n-i,phi+omega*2**i,itr]*fFunction(net_dict["L_{0}{1}{2}".format(n+1-i,psi+2*omega*2**(i-1),0)],net_dict["R_{0}{1}{2}".format(n-i,phi-1+omega*2**i,0)])

y_output = tf.zeros((1))
for i in range(N):
    if(FZlookup[i] == -1):
        y_output = tf.concat([y_output,net_dict["L_{0}{1}{2}".format(0,i,0)]],0)
y_output = tf.transpose(tf.reshape(y_output[1:],(K,batch_size)))*-1
loss = loss + 1.0*tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y_output,labels=y))

In [ ]:
best_val_ber = 1
load_weight = 0
quantize_weight = 0 #0 for non-quantize, 1 for normal, 2 for binarized, 3 for bin, 4 for binarized bin
bin_bit = 4  # number of different value
binary_prec = 4 # binary precision, binary_prec must >= bin_bit

if not (load_weight):
    train_step = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(loss)
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))  #allow tensorflow to automatically allocate device
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

if(quantize_weight == 0):
    weight_path = '{0}_{1}_{2}_{3}_{4}_{5}.ckpt'.format(N,ebn0[0],ebn0[-1],bp_iter_num,RNN,quantize_weight)
elif(quantize_weight == 1 or quantize_weight == 2):
    weight_path = '{0}_{1}_{2}_{3}_{4}_{5}_{6}.ckpt'.format(N,ebn0[0],ebn0[-1],bp_iter_num,RNN,quantize_weight,binary_prec)
elif(quantize_weight == 3):
    weight_path = '{0}_{1}_{2}_{3}_{4}_{5}_{6}.ckpt'.format(N,ebn0[0],ebn0[-1],bp_iter_num,RNN,quantize_weight,bin_bit)
else:
    weight_path = '{0}_{1}_{2}_{3}_{4}_{5}_{6}_{7}.ckpt'.format(N,ebn0[0],ebn0[-1],bp_iter_num,RNN,quantize_weight,bin_bit,binary_prec)    
    
if not(load_weight):
    for itr in range(50):
        train_nfails = 0
        train_loss = 0
        train_nframe = 0
        val_nfails = 0
        val_loss = 0
        # train
        wordRandom = np.random.RandomState(word_seed)
        noiseRandom = np.random.RandomState(noise_seed)        
        for i in range(batches_train):
            x_train, y_train = gendata(0,False,False)
            y_pred, _loss, _ = sess.run(fetches=[y_output, loss, train_step], feed_dict={x: x_train, y: y_train})
            train_loss = _loss + train_loss
            uhat = np.zeros((batch_size,K))
            uhat[y_pred>=0] = 1
            train_nfails = train_nfails + sum(sum(uhat!=y_train))
            train_nframe = train_nframe + sum(np.sum(uhat!=y_train,1)!=0)
        # quantize
        if(quantize_weight):
            LWeight, RWeight = get_weight(sess, LV, RV)
            if(quantize_weight == 1):
                LWeight = quantize(LWeight,binary_prec)
                RWeight = quantize(RWeight,binary_prec)
            elif(quantize_weight == 2):
                LWeight = quantizeToClosestBinary(LWeight,binary_prec)
                RWeight = quantizeToClosestBinary(RWeight,binary_prec)
            elif(quantize_weight == 3):
                LWeight = quantizeToBins(LWeight,bin_bit)
                RWeight = quantizeToBins(RWeight,bin_bit)
            else:
                LWeight = BINARYquantizeToBins(LWeight,bin_bit,binary_prec)
                RWeight = BINARYquantizeToBins(RWeight,bin_bit,binary_prec)            
            assign_weight(LWeight, RWeight, sess, LV, RV)
        # validation
        for i in range(batches_val):
            x_val, y_val = gendata(0,False,False)
            y_pred, _loss = sess.run(fetches=[y_output, loss], feed_dict={x: x_val, y: y_val})
            val_loss = _loss + val_loss
            uhat = np.zeros((batch_size,K))
            uhat[y_pred>=0] = 1
            val_nfails = val_nfails + sum(sum(uhat!=y_val))
        if(best_val_ber > (val_nfails/(batch_size*K*batches_val))):
            best_val_ber = val_nfails/(batch_size*K*batches_val)
            saver.save(sess, './weight.ckpt')
            Nobetter = 0
        else:
            Nobetter = Nobetter + 1
        if(Nobetter > patience):
            break    
        print('Epoch:\t{0:d}'.format(itr),'\tLoss:{0:.7f}'.format(train_loss/batches_train),'\tBER:{0:.7f}'.format(train_nfails/(batch_size*K*batches_train)),'\tFER:{0:.7f}'.format(train_nframe/(batch_size*batches_train)),'\tVal Loss:{0:.7f}'.format(val_loss/batches_val),'\tVal BER:{0:.7f}'.format(val_nfails/(batch_size*K*batches_val)))

# test
if(load_weight):
    saver.restore(sess, 'Weight/'+weight_path)
else:
    saver.restore(sess, './weight.ckpt')

test_nfails = np.zeros((len(ebn0)))
test_loss = np.zeros((len(ebn0)))
test_nframe = np.zeros((len(ebn0)))
wordRandom = np.random.RandomState(word_seed-200)
noiseRandom = np.random.RandomState(noise_seed-200) 
for j in range(len(ebn0)):
    for i in range(int(batches_test/len(ebn0))):
        x_test, y_test = gendata(j,True,False)
        y_pred, _loss = sess.run(fetches=[y_output, loss], feed_dict={x: x_test, y: y_test})
        test_loss[j] = _loss + test_loss[j]
        uhat = np.zeros((batch_size,K))
        uhat[y_pred>=0] = 1
        test_nfails[j] = test_nfails[j] + sum(sum(uhat!=y_test))
        test_nframe[j] = test_nframe[j] + sum(np.sum(uhat!=y_test,1)!=0)
print('Test SNR:  ',ebn0)
print('Test Loss: ',test_loss/int(batches_test/len(ebn0)))
print('Test BER:  ',test_nfails/(batch_size*K*(batches_test/len(ebn0))))
print('Test FER:  ',test_nframe/(batch_size*(batches_test/len(ebn0))))
if not(load_weight):
    saver.save(sess, 'Weight/'+weight_path)

In [ ]:
f_results = open('Results/'+weight_path[:-4]+'txt', 'w+')
f_results.write('Test SNR:    ' + str(ebn0) + '\n')
f_results.write('Test Loss:   ' + str(test_loss/int(batches_test/len(ebn0))) + '\n')
f_results.write('Test BER:    ' + str(test_nfails/(batch_size*K*(batches_test/len(ebn0)))) + '\n')
f_results.write('Test FER:    ' + str(test_nframe/(batch_size*(batches_test/len(ebn0)))) + '\n')
f_results.write('NumOfWord:   ' + str(numOfWord) + '\n')
f_results.write('Batch_size:  ' + str(batch_size) + '\n')
f_results.write('Batch_train: ' + str(batches_train) + '\n')
f_results.write('Batch_test:  ' + str(batches_test) + '\n')
f_results.write('Batch_val:   ' + str(batches_val) + '\n')
f_results.flush()